In [ ]:
#%pip install kagglehub
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [9]:
# Example
file_path = "written_name_test_v2.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "landlord/handwriting-recognition",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

C:\Users\tyson\AppData\Local\Temp\ipykernel_18892\2912609907.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


First 5 records:         FILENAME  IDENTITY
0  TEST_0001.jpg     KEVIN
1  TEST_0002.jpg  CLOTAIRE
2  TEST_0003.jpg      LENA
3  TEST_0004.jpg     JULES
4  TEST_0005.jpg   CHERPIN


In [ ]:
# Remove all existing TensorFlow installations
#%pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu protobuf keras-ocr
#%pip cache purge

# Install specific versions with --no-deps to prevent auto-upgrades
#%pip install tensorflow-gpu==2.10.0 --no-deps
#%pip install tensorflow-cpu==2.10.0 --no-deps
#%pip install tensorflow==2.10.0 --no-deps
#%pip install protobuf==3.20.3 --no-deps
#%pip install python-Levenshtein
#%pip install pytesseract
#%pip install easyocr
#%pip uninstall keras-ocr
#%pip install keras-ocr

# Pytesseract